In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/chinese task/CLUENER2020')

In [ ]:
#安装
!pip install transformers datasets

     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 290 kB 63.5 MB/s 
     |████████████████████████████████| 596 kB 69.0 MB/s 
     |████████████████████████████████| 895 kB 74.2 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
     |████████████████████████████████| 3.3 MB 17.0 MB/s 
     |████████████████████████████████| 243 kB 69.4 MB/s 
     |████████████████████████████████| 1.1 MB 36.5 MB/s 
     |████████████████████████████████| 132 kB 65.3 MB/s 
     |████████████████████████████████| 192 kB 74.6 MB/s 
     |████████████████████████████████| 160 kB 37.5 MB/s 
     |████████████████████████████████| 271 kB 72.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#定义全局超参数
batch_size=64
lr=1e-5
dropout=0.15
epoch=3


In [ ]:
import os
import json
import logging
import numpy as np
import config

In [ ]:
# 定义process函数，将原始标签替换为BIOS标签，并与token（字）一一对应
# 处理完的word_list BIOS_label_list存储为npz格式

def process(mode):
  data_dir='./data/'
  input_dir=data_dir+str(mode)+'.json'
  output_dir=data_dir+str(mode)+'.npz'
  if os.path.exists(output_dir) is True:
    return

  word_list=[]
  label_list=[]
  with open(input_dir,'r',encoding='utf-8') as f:
    # 先读取到内存中，然后逐行处理
    for line in f.readlines():
      # loads()：用于处理内存中的json对象，strip去除可能存在的空格
      json_line = json.loads(line.strip())

      text=json_line['text']
      words=list(text)
      # 如果没有label，则返回None
      label_entities = json_line.get('label', None)
      labels=['O']*len(words)

      if label_entities is not None:
        for key, value in label_entities.items():
          for sub_name, sub_index in value.items():
            for start_index, end_index in sub_index:
              assert ''.join(words[start_index:end_index+1]) == sub_name
              if start_index == end_index:
                labels[start_index]='S-'+ key
              else:
                labels[start_index]='B-'+ key
                labels[start_index+1:end_index+1]=['I-'+key]*(len(sub_name)-1)
      word_list.append(words)
      label_list.append(labels)
      # 保存成二进制文件
      np.savez_compressed(output_dir,words=word_list,labels=label_list)
      logging.info("--------{} data process DONE!--------".format(mode))

In [ ]:
#处理训练集和验证集数据
mode1,mode2,mode3='train','dev','test'
train_data=process(mode1)#45min处理时间
dev_data=process(mode2)
test_data=process(mode3)

"""
转换结果为：
"text": "浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，"
"label":['B-company', 'I-company', 'I-company', 'I-company', 'O', 'O', 'O', 'O', 'O', 'B-name', 'I-name', 'I-name', 
'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
 'B-name', 'I-name', 'I-name', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
"""

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


'\n转换结果为：\n"text": "浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，"\n"label":[\'B-company\', \'I-company\', \'I-company\', \'I-company\', \'O\', \'O\', \'O\', \'O\', \'O\', \'B-name\', \'I-name\', \'I-name\', \n\'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\',\n \'B-name\', \'I-name\', \'I-name\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\', \'O\']\n'

In [ ]:
#加载处理完的npz数据集
#不加allow_pickle=True会报错Object arrays cannot be loaded when allow_pickle=False，numpy新版本中默认为False。
train_data=np.load('./data/train.npz',allow_pickle=True)
val_data=np.load('./data/dev.npz',allow_pickle=True)
val_data.files

['words', 'labels']

In [3]:
import torch
a=torch.Tensor([1,2,-1,-1,-1,10])
b=a.gt(-1)#>-1的部分为真
c=(a!=-1)
c

tensor([ True,  True, False, False, False,  True])